In [559]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
import math
import pickle
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
np.random.seed(123)
#https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/
from sklearn.metrics import accuracy_score
from sklearn import linear_model, metrics, model_selection, ensemble
from sklearn.neural_network import MLPClassifier


In [565]:
predict_play = pickle.load(open('models/predict_play.sav', 'rb'))
rush_reg = pickle.load(open('models/rush_reg.sav', 'rb'))
pass_reg = pickle.load(open('models/pass_reg.sav', 'rb'))
rush_2 = pickle.load(open('models/rush_reg_2.sav', 'rb'))
fourth_down_classifier = pickle.load(open('models/4th_down_classifier.sav', 'rb'))
field_goal_classifier = pickle.load(open('models/field_goal_made.sav', 'rb'))
first_quarter_classifier = pickle.load(open('models/first_quarter_predict.sav', 'rb'))
second_quarter_classifier = pickle.load(open('models/second_quarter_predict.sav', 'rb'))
third_quarter_classifier = pickle.load(open('models/XGBThirdQuarter.sav', 'rb'))
fourth_quarter_classifier = pickle.load(open('models/XGBFourthQuarter.sav', 'rb'))

/usr/local/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator GaussianNB from version 0.22 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [429]:
# just a df to play with
df = pd.read_csv('./data/PBP - 2016 - Week 1.csv')
special_teams_stats = pd.read_csv('./data/Special Teams Stats 2016.csv')
special_teams_stats = special_teams_stats.drop(columns=['Rk'])
offense_stats = pd.read_csv('./data/Offense Stats 2016.csv')
defense_stats = pd.read_csv('./data/Defense Stats 2016.csv')

fulldf = df.join(special_teams_stats.set_index('School'), on='offenseTeam')
fulldf = fulldf.join(offense_stats.set_index('School'), on='offenseTeam', rsuffix='_offense_stats')
fulldf = fulldf.join(defense_stats.set_index('School'), on='defenseTeam', rsuffix='_defense_stats')
fulldf = fulldf.dropna()
fulldf

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,FG made,FG attempted,FG%,Pts kicking,Punts,Punting yds,average yards per punt,kickoff return yards,kickoff return yards per kickoff,punt returns,punt return yards,punt return yards per return,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards,Rk_defense_stats,Points Allowed,opponent completions,opponent attempts,opponent completion percentage,opponent passing yards,opponent passing touchdowns,opponent rush attempts,opponent rush yards,opponent average rush,opponent rush touchdowns,total opponent plays,total opponent yards,opponent total yards per play,opponent pass first downs,opponent rush first downs
374,400868910,2016,1,218,Temple,TEM,349,Army,ARMY,0,0,349,Army,ARMY,218,Temple,TEM,0,0,1,15:00,2016-09-03T06:02:56Z,Kickoff,1,10,65,0,25,Mitchell Howard kickoff for 65 yds for a touch...,0.5,0.9,50.0,4.5,2.8,100.0,35.1,40.4,16.9,1.6,11.2,7.0,59.0,13.0,29.9,4.2,9.5,43.9,74.4,0.5,60.5,339.6,5.6,3.5,70.0,414.0,5.9,2.9,17.8,4.8,42.2,11.0,18.4,13.0,24.4,53.2,152.1,0.9,36.4,130.5,3.6,1.2,60.8,282.6,4.6,6.9,7.2
375,400868910,2016,1,218,Temple,TEM,349,Army,ARMY,0,1,218,Temple,TEM,349,Army,ARMY,0,0,1,15:00,2016-09-03T06:02:56Z,Rush,1,10,25,6,31,Ryquell Armstead run for 6 yds to the Temp 31,1.8,2.1,86.2,9.1,3.8,147.5,39.0,43.8,21.9,1.3,13.6,10.6,43.0,14.0,32.4,16.6,28.8,57.6,237.4,1.6,40.1,176.4,4.4,2.2,68.9,413.8,6.0,9.7,9.9,5.4,43.1,15.0,19.8,14.7,26.0,56.5,170.2,1.1,31.2,121.2,3.9,1.3,57.2,291.5,5.1,7.0,6.7
376,400868910,2016,1,218,Temple,TEM,349,Army,ARMY,0,2,218,Temple,TEM,349,Army,ARMY,0,0,1,15:00,2016-09-03T06:02:56Z,Rush,2,4,31,2,33,Ryquell Armstead run for 2 yds to the Temp 33,1.8,2.1,86.2,9.1,3.8,147.5,39.0,43.8,21.9,1.3,13.6,10.6,43.0,14.0,32.4,16.6,28.8,57.6,237.4,1.6,40.1,176.4,4.4,2.2,68.9,413.8,6.0,9.7,9.9,5.4,43.1,15.0,19.8,14.7,26.0,56.5,170.2,1.1,31.2,121.2,3.9,1.3,57.2,291.5,5.1,7.0,6.7
377,400868910,2016,1,218,Temple,TEM,349,Army,ARMY,0,3,218,Temple,TEM,349,Army,ARMY,0,0,1,15:00,2016-09-03T06:02:56Z,Pass Reception,3,2,33,7,40,Phillip Walker pass complete to Marshall Ellic...,1.8,2.1,86.2,9.1,3.8,147.5,39.0,43.8,21.9,1.3,13.6,10.6,43.0,14.0,32.4,16.6,28.8,57.6,237.4,1.6,40.1,176.4,4.4,2.2,68.9,413.8,6.0,9.7,9.9,5.4,43.1,15.0,19.8,14.7,26.0,56.5,170.2,1.1,31.2,121.2,3.9,1.3,57.2,291.5,5.1,7.0,6.7
378,400868910,2016,1,218,Temple,TEM,349,Army,ARMY,0,4,218,Temple,TEM,349,Army,ARMY,0,0,1,15:00,2016-09-03T06:02:56Z,Pass Reception,1,10,40,7,47,Phillip Walker pass complete to Marshall Ellic...,1.8,2.1,86.2,9.1,3.8,147.5,39.0,43.8,21.9,1.3,13.6,10.6,43.0,14.0,32.4,16.6,28.8,57.6,237.4,1.6,40.1,176.4,4.4,2.2,68.9,413.8,6.0,9.7,9.9,5.4,43.1,15.0,19.8,14.7,26.0,56.5,170.2,1.1,31.2,121.2,3.9,1.3,57.2,291.5,5.1,7.0,6.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13351,400869507,2016,1,248,Houston,HOU,201,Oklahoma,OKLA,23,0,201,Oklahoma,OKLA,248,Houston,HOU,33,23,4,2:05,2016-09-04T02:25:41Z,Kickoff,2,10,65,0,53,Kyle Pfau on-side kick recovered by HOUSTON at...,0.8,1.2,68.8,8.1,3.7,151.9,41.1,62.7,23.3,0.9,9.4,10.2,3.0,13.0,43.9,20.8,29.5,70.6,318.0,3.3,44.0,236.8,5.4,2.3,73.5,554.8,7.5,13.5,10.5,5.1,52.2,30.0,23.5,18.1,32.2,56.1,219.5,1.5,34.4,100.2,2.9,1.2,66.6,319.6,4.8,9.5,5.7
13352,400869507,2016,1,248,Houston,HOU,201,Oklahoma,OKLA,23,1,248,Houston,HOU,201,Oklahoma,OKLA,33,23,4,2:00,2016-0

In [430]:
def seconds_left_in_quarter(clock):
    clock = clock.split(':')
    clock = (int(clock[0])*60) + int(clock[1])
    return clock

In [431]:
for index,row in fulldf.iterrows():
    fulldf.at[index,'clock'] = seconds_left_in_quarter(row['clock'])

In [103]:
sample_row = fulldf[35:36]
sample_row

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,FG made,FG attempted,FG%,Pts kicking,Punts,Punting yds,average yards per punt,kickoff return yards,kickoff return yards per kickoff,punt returns,punt return yards,punt return yards per return,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
53,400868876,2016,1,239,Baylor,BAY,2466,Northwestern St,NWST,8,0,239,Baylor,BAY,2466,Northwestern St,NWST,24,0,1,62,2016-09-03T07:28:14Z,Kickoff,1,0,35,0,90,"Drew Galitz kickoff for 65 yds , Shakeir Ryan ...",0.8,1.2,62.5,6.8,4.4,181.5,41.4,51.5,19.1,1.8,17.2,9.7,35.0,13.0,34.6,20.7,36.9,56.0,280.9,2.5,48.3,241.8,5.0,1.9,85.2,522.7,6.1,10.8,13.4,9.9,82.2


In [94]:
def get_down(row):
    return int(row['down'])

In [288]:
play_and_result(sample_row)

Rush


(2, 5.573473177019568, 53    3.940472
 Name: distance, dtype: float64)

In [202]:
def update_position(sample_row, old_down, calc_distance, old_distance):
    new_row = sample_row
    new_distance = old_distance - calc_distance
    new_yard_line = int(new_row['yardLine']) + calc_distance
    if(new_distance < 0):
        new_row['down'] = 1
        # need to check if it's close to the goal line :/
        new_row['distance'] = 10
        new_row['yardLine'] = new_yard_line
    else:
        old_down+=1
        new_row['distance'] = new_distance
        new_row['down'] = old_down
        new_row['yardLine'] = new_yard_line
    return new_row

In [120]:
old_down, calc_distance, old_distance = play_and_result(sample_row)
new_row = update_position(sample_row, old_down, calc_distance, old_distance)

not 4th
Rush


/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [217]:
plays = []
for index,row in fulldf.iterrows():
    new_row = row
    down = get_down(new_row)
    i = 0
    while (new_row['yardLine'] < 100 and down != 4):
        i+=1
        print(i)
        old_down, calc_distance, old_distance = play_and_result(new_row)
        new_row = update_position(new_row, old_down, calc_distance, old_distance)
        down = get_down(new_row)
    plays.append(i)

1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
1
not 4th
Pass
2
not 4th
Rush
3
not 4th
Rush
1
not 4th

18
not 4th
Rush
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Pass
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rus

Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rush
1
not 4th
Rush
2
not 4th
Pass
3
not 4th
Pass
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rush
1
not 4th
Pass
2
not 4th
Pass
1
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
1
not 4th
Rush
2
not 4th
Pass
3
not 4th
Pass
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Pass
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4t

Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
17
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
16
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
14
not 4th
Rush
15
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th
Rush
8
not 4th
Rush
9
not 4th
Rush
10
not 4th
Rush
11
not 4th
Rush
12
not 4th
Rush
13
not 4th
Rush
1
not 4th
Rush
2
not 4th
Rush
3
not 4th
Rush
4
not 4th
Rush
5
not 4th
Rush
6
not 4th
Rush
7
not 4th


KeyboardInterrupt: 

In [234]:
def fourth_down_play(row):
    features = row[['homeScore', 'awayScore', 'quarter', 'distance', 
                    'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps', 'completion percentage', 
                    'passing yards', 'Passing Touchdowns', 'Rush Attempts', 'Rushing yards', 'rush average', 
                    'Rush Touchdowns', 'Total Offense Plays', 'Total yards', 'total yards per play', 
                    'First down by pass', 'First down by rush']].values.reshape(1, -1)
    result = fourth_down_classifier.predict(features)[0]
    #test_labels = test_labels.replace('Punt', 0)
    # test_labels = test_labels.replace('Field Goal', 1)
    # test_labels = test_labels.replace('Pass', 2)
    # test_labels = test_labels.replace('Rush', 3)
    if(result == 0):
        return 'Punt'
    elif result ==1:
        return 'Field Goal'
    elif result == 2:
        return 'Pass'
    else:
        return 'Rush'

In [241]:
def field_goal_result(row):
    features = row[['yardLine', 'FG made', 'FG%']].values.reshape(1,-1)
    result = field_goal_classifier.predict(features)[0]
#         if(df_type == 'Field Goal Good'):
#         return 1
#     else:
#         return 0
    if(result == 1):
        return 'Field Goal Good'
    else:
        return 'Missed'

In [242]:
field_goal_result(test_row)

'Missed'

In [448]:
def fourth_down(test_row):
    play = fourth_down_play(test_row)
    if play == 'Field Goal':
        result = field_goal_result(test_row)
        if(result == 'Field Goal Good'):
            # add 3 points
            return 3
        else:
            return 'break'
    elif play == 'Pass':
        feature = test_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts',
                            'Passing Completion', 'Pass Attemps',
                            'completion percentage', 'passing yards', 'Passing Touchdowns',
                            'Total Offense Plays', 'Total yards', 'total yards per play',
                            'First down by pass', 'First down by rush']].values.reshape(1,-1)
        yards_gained = pass_reg.predict(feature)
        if(test_row['yardLine'] + yards_gained >= 100):
            return 7
        elif(yards_gained >= test_row['distance']):
            return yards_gained
        else:
            return 'break'
    elif play == 'Rush':
        feature = test_row[[
       'down', 'distance', 'yardLine',
       'Rush Attempts', 'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
        'First down by rush', 'Points Allowed', 'opponent rush attempts',
       'opponent rush yards', 'opponent average rush',
       'opponent rush touchdowns', 'total opponent plays',
       'total opponent yards', 'opponent total yards per play',
       'opponent pass first downs', 'opponent rush first downs']].values.reshape(1,-1)
        yards_gained = rush_2.predict(feature)
        if(test_row['yardLine'] + yards_gained >= 100):
            return 7
        elif(yards_gained >= test_row['distance']):
            return yards_gained
        else:
            return 'break'
    elif play == 'Punt':
        return 'break'


In [264]:
fourth_down(test_row)

'break'

In [447]:
# check if 4th down
def play_and_result(sample_row):
    if(get_down(sample_row) == 4):
        result = fourth_down(sample_row)
        if (result ==3 or result ==7 or result == 'break'):
            print('end ' + str(result))
            return 0, result, result
        else:
            print('break continue')
            print(result)
            return 0, result, sample_row['distance']
    else:
        feature = sample_row[['quarter', 'down', 'distance', 'yardLine', 'clock']]
        feature = feature.values.reshape(1, -1)
        play = predict_play.predict(feature)[0]
        print(play)
        if(play == 'Rush'):
            feature = sample_row[[
       'down', 'distance', 'yardLine',
       'Rush Attempts', 'Rushing yards', 'rush average', 'Rush Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
        'First down by rush', 'Points Allowed', 'opponent rush attempts',
       'opponent rush yards', 'opponent average rush',
       'opponent rush touchdowns', 'total opponent plays',
       'total opponent yards', 'opponent total yards per play',
       'opponent pass first downs', 'opponent rush first downs']]
            feature = feature.values.reshape(1,-1)
            yards_gained = rush_2.predict(feature)
            return get_down(sample_row), yards_gained[0], sample_row['distance']
        else:
            feature = sample_row[['down', 'distance', 'yardLine', 'Rk', 'G', 'Pts', 'Passing Completion', 'Pass Attemps',
       'completion percentage', 'passing yards', 'Passing Touchdowns',
       'Total Offense Plays', 'Total yards', 'total yards per play',
       'First down by pass', 'First down by rush']]
            feature = feature.values.reshape(1,-1)
            yards_gained = pass_reg.predict(feature)
            return get_down(sample_row), yards_gained[0], sample_row['distance']
    # if it's not 4th down take the position and put it into predict_play

    # if the result is a run use the rush_reg, else use the pass_reg

    # get the result of that and update the field position, increment the down if necessary

    # repeat until score or 4th down

In [273]:
result = play_and_result(test_row)

Rush


(1, 5.040928217403135, 10)

In [286]:
old_down, calc_distance, old_distance = play_and_result(test_row)
new_row = update_position(new_row, old_down, calc_distance, old_distance)

Rush


In [389]:
not_touchdown =[]
no_score = []
field_goals = []
for index,row in fulldf.iterrows():
    new_row = row
    while True:
        old_down, calc_distance, old_distance = play_and_result(new_row)
        if(calc_distance == 'break'):
            no_score.append(new_row)
        if(calc_distance == 3):
            field_goals.append(new_row)
        if(calc_distance == 3 or calc_distance == 7 or calc_distance == 'break' or new_row['yardLine'] > 100):
            if(new_row['yardLine'] < 100):
                not_touchdown.append(new_row)
            break
        new_row = update_position(new_row, old_down, calc_distance, old_distance)

break end
break end
break end
break end
break end
break end
break end
break end
break end
break continue
[5.92590911]
break end
break end
break end
break end


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':


break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end
break end


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/pytho

break continue
[6.24745984]
break end
break continue
[6.26298]
break end
break end
break end
break end
break continue
[4.89740247]
break end
break end
break end
break end
break end
break end
break end
break end
break end
break continue
[4.57159729]
break end
break end
break continue
[5.25820396]
break continue
[4.76314465]
break end


...

/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/pytho

break continue
[3.63812883]
break end
break end
break end
break continue
[4.89679642]
break continue
[4.89679642]
break end
break continue
[5.21205501]
break end
break end
break end
break end
break continue
[3.58793069]
break continue
[3.55355572]
break continue
[3.30191181]
break continue
[3.30191181]
break continue
[4.75730473]
break end
break end
break end
break end
break end
break end
break continue
[5.56372774]
break end
break end
break continue
[6.22152704]
break continue
[5.7565673]
break continue
[5.7565673]


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/pytho

break continue
[5.55224173]


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':


In [395]:
len(no_score) #769
no_score[90]

gameId                                                                     400868969
year                                                                            2016
week                                                                               1
homeId                                                                           333
homeTeam                                                                     Alabama
homeAbbr                                                                         ALA
awayId                                                                            30
awayTeam                                                                         USC
awayAbbr                                                                         USC
driveIndex                                                                        12
playIndex                                                                          0
offenseId                                                        

In [391]:
len(not_touchdown) #1526
# not_touchdown[1063]

1526

In [359]:
new_row

gameId                                                                      400869094
year                                                                             2016
week                                                                                1
homeId                                                                             24
homeTeam                                                                     Stanford
homeAbbr                                                                         STAN
awayId                                                                           2306
awayTeam                                                                 Kansas State
awayAbbr                                                                          KSU
driveIndex                                                                          4
playIndex                                                                           0
offenseId                                             

In [345]:
fulldf[127:135]

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,FG made,FG attempted,FG%,Pts kicking,Punts,Punting yds,average yards per punt,kickoff return yards,kickoff return yards per kickoff,punt returns,punt return yards,punt return yards per return,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards
247,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,11,7,66,Iowa State,ISU,2460,Northern Iowa,UNI,7,7,2,506,2016-09-04T08:10:14Z,Pass Incompletion,3,16,36,0,36,"Joel Lanning pass incomplete to Allen Lazard, ...",1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
248,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,11,8,66,Iowa State,ISU,2460,Northern Iowa,UNI,7,7,2,506,2016-09-04T08:10:14Z,Punt,4,16,36,0,84,"Colin Downing punt for 48 yds, fair catch by D...",1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
255,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,0,66,Iowa State,ISU,2460,Northern Iowa,UNI,7,7,2,284,2016-09-04T08:20:40Z,Penalty,1,10,51,-10,41,"Lanning, Joel pass complete to Ryen, Trever fo...",1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
256,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,7,7,2,284,2016-09-04T08:20:40Z,Pass Reception,1,20,41,42,83,Joel Lanning pass complete to Allen Lazard for...,1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
257,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,2,66,Iowa State,ISU,2460,Northern Iowa,UNI,7,7,2,284,2016-09-04T08:20:40Z,Sack,1,10,83,-1,82,Joel Lanning sacked by Hezekiah Applegate for ...,1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
258,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,3,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,7,2,200,2016-09-04T08:23:23Z,Passing Touchdown,2,11,82,18,100,Joel Lanning pass complete to Hakeem Butler fo...,1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
259,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,14,0,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,7,2,200,2016-09-04T08:23:23Z,Kickoff,1,0,35,28,72,"Chris Francis kickoff for 65 yds , Michael Mal...",1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8
264,400868887,2016,1,66,Iowa State,ISU,2460,Northern Iowa,UNI,15,0,66,Iowa State,ISU,2460,Northern Iowa,UNI,13,7,2,132,2016-09-04T08:30:18Z,Rush,1,10,11,0,11,Mike Warren run for no gain to the IowSt 11,1.3,1.4,94.1,7.0,4.8,198.4,41.1,84.8,23.1,2.0,18.8,9.4,71.0,12.0,27.7,19.6,33.2,59.0,258.8,1.8,38.2,162.8,4.3,1.5,71.3,421.6,5.9,11.1,8.6,6.3,56.8


In [458]:
new_row = fulldf.iloc[249]
print(new_row['yardLine'])
while True:
    old_down, calc_distance, old_distance = play_and_result(new_row)
    print(calc_distance)
    if(calc_distance == 3 or calc_distance == 7 or calc_distance == 'break' or new_row['yardLine'] + calc_distance > 100):
        if(new_row['yardLine'] + calc_distance > 100 or calc_distance == 7):
            if(new_row['offenseTeam'] == new_row['homeTeam']):
                new_row['homeScore'] += 7
            elif(new_row['offenseTeam'] == new_row['awayTeam']):
                new_row['awayTeam'] += 7
        elif(calc_distance == 3):
            if(new_row['offenseTeam'] == new_row['homeTeam']):
                new_row['homeScore'] += 3
            elif(new_row['offenseTeam'] == new_row['awayTeam']):
                new_row['awayTeam'] += 3
        print(new_row['yardLine'] + calc_distance)
        break
    new_row = update_position(new_row, old_down, calc_distance, old_distance)

99
Rush
0.7264673552867821
Rush
0.7264673552867821
100.45293471057357


/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [566]:
def predict_winner(new_row):
    features = new_row[['homeScore', 'awayScore', 'down', 'distance', 'Passing Completion',
       'Pass Attemps', 'completion percentage', 'passing yards',
       'Passing Touchdowns', 'Rush Attempts', 'Rushing yards', 'rush average',
       'Rush Touchdowns', 'Total Offense Plays', 'Total yards',
       'total yards per play', 'First down by pass', 'First down by rush',
       'Number of Penalties', 'opponent completions', 'opponent attempts',
       'opponent completion percentage', 'opponent passing yards',
       'opponent passing touchdowns', 'opponent rush attempts',
       'opponent rush yards', 'opponent average rush',
       'opponent rush touchdowns', 'total opponent plays',
       'total opponent yards', 'opponent total yards per play',
       'opponent pass first downs', 'opponent rush first downs']]
    if(new_row['quarter'] == 4):
        if(new_row['offenseTeam'] == new_row['homeTeam']):
            return fourth_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][1]
        else:
            return fourth_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][0]
    elif (new_row['quarter'] == 3):
        if(new_row['offenseTeam'] == new_row['homeTeam']):
            return third_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][1]
        else:
            return third_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][0]
    elif (new_row['quarter']==2):
        if(new_row['offenseTeam'] == new_row['homeTeam']):
            return second_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][1]
        else:
            return second_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][0]
    else:
        if(new_row['offenseTeam'] == new_row['homeTeam']):
            return first_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][1]
        else:
            return first_quarter_classifier.predict_proba(features.values.reshape(1,-1))[0][0]


In [567]:
predict_winner(new_row)

0.9109974031693837

In [459]:
new_row

gameId                                                                  400868946
year                                                                         2016
week                                                                            1
homeId                                                                        251
homeTeam                                                                    Texas
homeAbbr                                                                      TEX
awayId                                                                         87
awayTeam                                                               Notre Dame
awayAbbr                                                                       ND
driveIndex                                                                      9
playIndex                                                                       7
offenseId                                                                     251
offenseTeam     

In [557]:
fulldf.iloc[248:378]

,gameId,year,week,homeId,homeTeam,homeAbbr,awayId,awayTeam,awayAbbr,driveIndex,playIndex,offenseId,offenseTeam,offenseAbbr,defenseId,defenseTeam,defenseAbbr,homeScore,awayScore,quarter,clock,wallclock,type,down,distance,yardLine,yardsGained,endYardLine,description,FG made,FG attempted,FG%,Pts kicking,Punts,Punting yds,average yards per punt,kickoff return yards,kickoff return yards per kickoff,punt returns,punt return yards,punt return yards per return,Rk,G,Pts,Passing Completion,Pass Attemps,completion percentage,passing yards,Passing Touchdowns,Rush Attempts,Rushing yards,rush average,Rush Touchdowns,Total Offense Plays,Total yards,total yards per play,First down by pass,First down by rush,Number of Penalties,Penalty yards,Rk_defense_stats,Points Allowed,opponent completions,opponent attempts,opponent completion percentage,opponent passing yards,opponent passing touchdowns,opponent rush attempts,opponent rush yards,opponent average rush,opponent rush touchdowns,total opponent plays,total opponent yards,opponent total yards per play,opponent pass first downs,opponent rush first downs
622,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,9,6,251,Texas,TEX,87,Notre Dame,ND,14,14,2,130,2016-09-05T07:25:18Z,Rush,1,1,99,0,99,Chris Warren III run for no gain to the NDame 1,1.4,2.0,70.8,7.6,5.5,259.6,47.2,64.4,18.4,2.3,15.2,6.5,46.0,12.0,31.9,20.1,33.5,60.0,252.0,1.8,49.6,240.2,4.8,2.3,83.1,492.2,5.9,10.2,13.1,7.1,64.1,62.0,27.8,16.1,26.1,61.7,196.4,1.2,44.1,182.4,4.1,1.9,70.2,378.8,5.4,8.5,10.0
623,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,9,7,251,Texas,TEX,87,Notre Dame,ND,14,14,2,111,2016-09-05T08:03:17Z,Rush,2,1,99,0,99,Tyrone Swoopes run for no gain to the NDame 1,1.4,2.0,70.8,7.6,5.5,259.6,47.2,64.4,18.4,2.3,15.2,6.5,46.0,12.0,31.9,20.1,33.5,60.0,252.0,1.8,49.6,240.2,4.8,2.3,83.1,492.2,5.9,10.2,13.1,7.1,64.1,62.0,27.8,16.1,26.1,61.7,196.4,1.2,44.1,182.4,4.1,1.9,70.2,378.8,5.4,8.5,10.0
624,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,9,8,251,Texas,TEX,87,Notre Dame,ND,21,14,2,72,2016-09-05T07:25:18Z,Rushing Touchdown,3,0,99,1,100,"Tyrone Swoopes run for 1 yd for a TD, (Trent D...",1.4,2.0,70.8,7.6,5.5,259.6,47.2,64.4,18.4,2.3,15.2,6.5,46.0,12.0,31.9,20.1,33.5,60.0,252.0,1.8,49.6,240.2,4.8,2.3,83.1,492.2,5.9,10.2,13.1,7.1,64.1,62.0,27.8,16.1,26.1,61.7,196.4,1.2,44.1,182.4,4.1,1.9,70.2,378.8,5.4,8.5,10.0
625,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,10,0,251,Texas,TEX,87,Notre Dame,ND,21,14,2,72,2016-09-05T08:08:20Z,Kickoff,3,10,35,0,75,Mitchell Becker kickoff for 65 yds for a touch...,1.4,2.0,70.8,7.6,5.5,259.6,47.2,64.4,18.4,2.3,15.2,6.5,46.0,12.0,31.9,20.1,33.5,60.0,252.0,1.8,49.6,240.2,4.8,2.3,83.1,492.2,5.9,10.2,13.1,7.1,64.1,62.0,27.8,16.1,26.1,61.7,196.4,1.2,44.1,182.4,4.1,1.9,70.2,378.8,5.4,8.5,10.0
626,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,10,1,87,Notre Dame,ND,251,Texas,TEX,21,14,2,63,2016-09-05T08:12:10Z,Pass Reception,1,10,75,14,61,DeShone Kizer pass complete to Josh Adams for ...,1.1,1.4,76.5,6.9,4.8,202.3,42.6,73.1,23.1,1.8,15.9,9.1,53.0,12.0,30.9,18.7,32.3,57.7,254.3,2.3,36.5,163.3,4.5,1.5,68.8,417.6,6.1,11.0,8.9,6.5,53.2,91.0,31.5,20.5,33.8,60.7,258.5,1.8,45.7,190.3,4.2,1.9,79.4,448.8,5.7,10.7,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,31,2,251,Texas,TEX,87,Notre Dame,ND,44,47,6,0,2016-09-05T10:31:41Z,Rush,3,1,84,10,94,Tyrone Swoopes run for 10 yds to the NDame 6 f...,1.4,2.0,70.8,7.6,5.5,259.6,47.2,64.4,18.4,2.3,15.2,6.5,46.0,12.0,31.9,20.1,33.5,60.0,252.0,1.8,49.6,240.2,4.8,2.3,83.1,492.2,5.9,10.2,13.1,7.1,64.1,62.0,27.8,16.1,26.1,61.7,196.4,1.2,44.1,182.4,4.1,1.9,70.2,378.8,5.4,8.5,10.0
748,400868946,2016,1,251,Texas,TEX,87,Notre Dame,ND,31,3,251,Texas,TEX,87,Notre Dame,ND